In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r"C:\Users\mhema\OneDrive\Desktop\DataScience\MachineLearning\Artifacts\MissingValueDataset.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550068 entries, 0 to 550067
Data columns (total 12 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   User_ID                     550068 non-null  int64  
 1   Product_ID                  550068 non-null  object 
 2   Gender                      550068 non-null  object 
 3   Age                         550068 non-null  object 
 4   Occupation                  550068 non-null  int64  
 5   City_Category               550068 non-null  object 
 6   Stay_In_Current_City_Years  550068 non-null  object 
 7   Marital_Status              550068 non-null  int64  
 8   Product_Category_1          550068 non-null  int64  
 9   Product_Category_2          376430 non-null  float64
 10  Product_Category_3          166821 non-null  float64
 11  Purchase                    550068 non-null  int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 50.4+ MB


In [4]:
selected_df = df[['Gender', 'Age', 'Occupation', 'City_Category', 'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category_1', 'Purchase']]

In [57]:
selected_df[['Marital_Status', 'Product_Category_1', 'Purchase']].corr()

,Marital_Status,Product_Category_1,Purchase
Marital_Status,1.000000,0.019888,-0.000463
Product_Category_1,0.019888,1.000000,-0.343703
Purchase,-0.000463,-0.343703,1.000000


In [5]:
selected_df.head()

,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Purchase
0,F,0-17,10,A,2,0,3,8370
1,F,0-17,10,A,2,0,1,15200
2,F,0-17,10,A,2,0,12,1422
3,F,0-17,10,A,2,0,12,1057
4,M,55+,16,C,4+,0,8,7969


- Gender <-> OHE
- Age <-> LE
- City_Category <-> OHE
- Stay_In_Current_City_Years <-> LE

In [28]:
selected_df['Purchase']

0          8370
1         15200
2          1422
3          1057
4          7969
          ...  
550063      368
550064      371
550065      137
550066      365
550067      490
Name: Purchase, Length: 550068, dtype: int64

In [37]:
from sklearn.model_selection import train_test_split

X = selected_df.drop(columns=['Purchase'])
y = selected_df['Purchase']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [38]:
X_train

,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1
178247,M,51-55,7,B,3,1,1
196647,M,18-25,12,C,4+,0,5
418590,M,26-35,5,B,1,0,8
408727,M,18-25,4,B,2,0,8
216416,M,18-25,4,B,1,0,1
...,...,...,...,...,...,...,...
110268,M,46-50,17,C,2,0,5
259178,M,26-35,14,C,2,1,5
365838,F,18-25,1,B,1,0,5
131932,F,18-25,4,B,2,0,8


In [33]:
X_test

,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1
84432,M,51-55,19,C,3,0,1
72724,F,36-45,11,B,1,0,1
197032,F,36-45,0,A,2,1,8
353704,M,18-25,4,A,3,0,1
91198,M,18-25,4,B,0,0,6
...,...,...,...,...,...,...,...
133324,M,55+,13,B,2,1,1
138718,F,26-35,4,B,3,0,8
22886,M,36-45,20,A,2,1,8
137110,M,36-45,7,A,0,0,11


In [21]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler

In [24]:
nominal_features = ['Gender', 'City_Category']
ordinal_features = ['Age', 'Stay_In_Current_City_Years']
numerical_columns = ['Occupation', 'Marital_Status', 'Product_Category_1']

In [17]:
df['Stay_In_Current_City_Years'].value_counts()

Stay_In_Current_City_Years
1     193821
2     101838
3      95285
4+     84726
0      74398
Name: count, dtype: int64

In [25]:
preprocessor = ColumnTransformer(
    transformers=[
        ('ohe', OneHotEncoder(), nominal_features),
        ('le', OrdinalEncoder(), ordinal_features),
        ('ss', StandardScaler(), numerical_columns)
    ],
)

In [36]:
preprocessor

ColumnTransformer(transformers=[('ohe', OneHotEncoder(),
                                 ['Gender', 'City_Category']),
                                ('le', OrdinalEncoder(),
                                 ['Age', 'Stay_In_Current_City_Years']),
                                ('ss', StandardScaler(),
                                 ['Occupation', 'Marital_Status',
                                  'Product_Category_1'])])

In [65]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
# from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor())
])

In [66]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('ohe', OneHotEncoder(),
                                                  ['Gender', 'City_Category']),
                                                 ('le', OrdinalEncoder(),
                                                  ['Age',
                                                   'Stay_In_Current_City_Years']),
                                                 ('ss', StandardScaler(),
                                                  ['Occupation',
                                                   'Marital_Status',
                                                   'Product_Category_1'])])),
                ('regressor',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, c...
                              feature_types=None, feature_weights=None,
                              gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=None, n_jobs=None,
                              num_parallel_tree=None, ...))])

In [67]:
X_test

,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Purchase,Prediction,Actual_Error,Absolute_Actual_Error,Absolute_Percentage_Error
84432,M,51-55,19,C,3,0,1,19142,14886.307379,4255.692621,4255.692621,0.222322
72724,F,36-45,11,B,1,0,1,15513,13376.045276,2136.954724,2136.954724,0.137753
197032,F,36-45,0,A,2,1,8,7802,8285.989904,-483.989904,483.989904,0.062034
353704,M,18-25,4,A,3,0,1,15455,13076.182544,2378.817456,2378.817456,0.153919
91198,M,18-25,4,B,0,0,6,4492,15919.118780,-11427.118780,11427.118780,2.543882
...,...,...,...,...,...,...,...,...,...,...,...,...
133324,M,55+,13,B,2,1,1,19139,15508.215087,3630.784913,3630.784913,0.189706
138718,F,26-35,4,B,3,0,8,7856,7311.149712,544.850288,544.850288,0.069355
22886,M,36-45,20,A,2,1,8,4037,6951.692040,-2914.692040,2914.692040,0.721995
137110,M,36-45,7,A,0,0,11,7467,5889.073251,1577.926749,1577.926749,0.211320


In [68]:
X_test['Purchase'] = y_test

In [69]:
X_test['Prediction'] = pipeline.predict(X_test)

In [70]:
X_test

,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Purchase,Prediction,Actual_Error,Absolute_Actual_Error,Absolute_Percentage_Error
84432,M,51-55,19,C,3,0,1,19142,16026.744141,4255.692621,4255.692621,0.222322
72724,F,36-45,11,B,1,0,1,15513,13530.408203,2136.954724,2136.954724,0.137753
197032,F,36-45,0,A,2,1,8,7802,7254.826172,-483.989904,483.989904,0.062034
353704,M,18-25,4,A,3,0,1,15455,13057.036133,2378.817456,2378.817456,0.153919
91198,M,18-25,4,B,0,0,6,4492,15936.153320,-11427.118780,11427.118780,2.543882
...,...,...,...,...,...,...,...,...,...,...,...,...
133324,M,55+,13,B,2,1,1,19139,15145.119141,3630.784913,3630.784913,0.189706
138718,F,26-35,4,B,3,0,8,7856,7430.262207,544.850288,544.850288,0.069355
22886,M,36-45,20,A,2,1,8,4037,6781.355957,-2914.692040,2914.692040,0.721995
137110,M,36-45,7,A,0,0,11,7467,4518.012695,1577.926749,1577.926749,0.211320


In [71]:
X_test['Actual_Error'] = X_test['Purchase'] - X_test['Prediction']
X_test['Absolute_Actual_Error'] = abs(X_test['Actual_Error'])
X_test['Absolute_Percentage_Error'] = X_test['Absolute_Actual_Error'] / X_test['Purchase']

In [72]:
X_test['Absolute_Percentage_Error'].mean()

0.37256955693467236